<a href="https://colab.research.google.com/github/dantae74/machine-learning/blob/main/stock-etc/03-02-get-corporate-open-information-from-dart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## DART 사이트에서 (삼성전자: 005930) 기업 공시 정보 가져오기 
1999-01-01부터 현재까지 분기/반기 보고서 가져와서 로컬 디렉토리에 csv로 저장하기

In [ ]:
import requests
from io import BytesIO
import pandas as pd
import re

In [ ]:
# crtfc_key = ""
# with open('./sample_data/my_data.txt', 'r') as f:
#   crtfc_key = f.readline()
  # print(crtfc_key)

In [ ]:
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"

In [ ]:
from datetime import datetime
now = datetime.today().strftime('%Y%m%d')

In [ ]:
corp_code = '005930' # 삼성전자

In [ ]:
def make_api_url(corp_code, bgn_de='19990101', end_de=now, pblntf_ty='A', pblntf_detail_ty=['A001','A002','A003'], page_count='100'):

  for i, code in enumerate(pblntf_detail_ty):
    pblntf_detail_ty[i] = 'pblntf_detail_ty=' + code
    if i != (len(pblntf_detail_ty)-1): pblntf_detail_ty[i] += '&'
  pblntf_detail_ty = "".join(pblntf_detail_ty)

  url = "https://opendart.fss.or.kr/api/list.json?crtfc_key={crtfc_key}&corp_code={corp_code}&bgn_de={bgn_de}&end_de={end_de}&pblntf_ty={pblntf_ty}&{pblntf_detail_ty}&page_count={page_count}".format(\
                                                              crtfc_key=crtfc_key,corp_code=corp_code,bgn_de=bgn_de,end_de=end_de,pblntf_ty=pblntf_ty,pblntf_detail_ty=pblntf_detail_ty,page_count=page_count)
  return url

In [ ]:
url = make_api_url(corp_code)
# print(url)

In [ ]:
response = requests.get(url, headers={"user-agent":user_agent})

In [ ]:
df_resp = pd.read_json(response.content)
df_resp

In [ ]:
list_resp = df_resp['list'].tolist()
df = pd.DataFrame.from_records(list_resp)
df

In [ ]:
report_nms = df['report_nm'].tolist()[:10]
rcept_nos = df['rcept_no'].tolist()[:10]

In [ ]:
dcm_nos = []
for rcept_no in rcept_nos:
  url =  'http://dart.fss.or.kr/dsaf001/main.do?rcpNo={rcept_no}'.format(rcept_no=rcept_no)
  response = requests.get(url, headers={"user-agent":user_agent})
  dcm_nos.append(re.search(r"'{rcept_no}', '(.*?)',".format(rcept_no=rcept_no), response.content.decode('UTF-8')).group(1))

In [ ]:
sheets = ["연결 재무상태표","연결 손익계산서","연결 포괄손익계산서","연결 자본변동표","연결 현금흐름표","재무상태표","손익계산서"]

def download_excel(report_nm, rcept_no, dcm_no):
    url = "http://dart.fss.or.kr/pdf/download/excel.do?rcp_no={rcp_no}&dcm_no={dcm_no}&lang=ko".format(rcp_no=rcept_no,dcm_no=dcm_no)
    response = requests.get(url, headers={"user-agent":user_agent})
    f = BytesIO(response.content)

    for sheet in sheets:
        df = pd.read_excel(f, sheet_name=sheet, skiprows=5)
        df.to_csv(report_nm+"_"+sheet+".csv", encoding="euc-kr")

In [ ]:
for report_nm, rcept_no, dcm_no in zip(report_nms, rcept_nos, dcm_nos):
  download_excel(report_nm, rcept_no, dcm_no)

In [ ]:
pd.read_csv('분기보고서 (2021.03)_재무상태표.csv',encoding="euc-kr")